In [4]:
#Este archivo es un webscrapping de la empresa de ropa HyM 

import requests
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import pdb
from datetime import date
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By


In [6]:
# pd.DataFrame(
#             lista_diccionarios,
#             index=range(len(lista_diccionarios))
#             ).to_excel('H&M_Novedades.xlsx', index=False)

In [8]:
def HyM(urls):

    lista_diccionarios = []
    for url in urls:
        n_item = 0
        driver.get(url)
        time.sleep(5)
        products_button_path = './/a[@id="menu-product"]'
        products_button = driver.find_element(By.XPATH, products_button_path)
        driver.execute_script("arguments[0].click();", products_button)
        grilla = driver.find_elements(By.XPATH, '//div[@id="gallery-layout-container"]')
        
        while True:
            try:
                # ================== Altura ================== #
                n_item += 1
                x_path = f'//div[@id="gallery-layout-container"]/div[{n_item}]'
                altura = driver.find_element("xpath", 
                            x_path).rect['y']
                driver.execute_script(f"window.scrollTo(0, {altura});")

                # ================== 1. item  ================== #
                for attempt in range(10):
                    try:
                        path_item = f'{x_path}//a[@class = "vtex-product-summary-2-x-clearLink vtex-product-summary-2-x-clearLink--grilla h-100 flex flex-column"]'
                        item = driver.find_element(By.XPATH, 
                                    path_item).get_attribute('href')
                    except:
                        print(f"No se pudo encontrar el item {n_item} en el attempt {attempt}")
                        time.sleep(1)
                        continue
                    else:
                        break
                
                # ================== 2. Foto ================== #
                for attempt in range(10):
                    try:
                        path_foto = f'{x_path}//div[@class="dib relative vtex-product-summary-2-x-imageContainer vtex-product-summary-2-x-imageContainer--commonImage vtex-product-summary-2-x-imageStackContainer vtex-product-summary-2-x-hoverEffect"]/img'
                        foto = driver.find_element(By.XPATH, 
                                    path_foto).get_attribute('src')
                    except:
                        print(f"No se pudo encontrar la foto {item} en el attempt {attempt}")
                        time.sleep(1)
                        continue
                    else:
                        break

                # ================== 3. Nombre ================== #
                for attempt in range(10):
                    try:
                        path_nombre = f'{x_path}//span[@class="vtex-product-summary-2-x-productBrand vtex-product-summary-2-x-brandName t-body"]'
                        nombre = driver.find_element(By.XPATH, 
                                    path_nombre).text
                    except:
                        print(f"No se pudo encontrar el nombre {item} en el attempt {attempt}")
                        time.sleep(1)
                        continue
                    else:
                        break

                # ================== 4. Precio actual ================== #
                for attempt in range(10):
                    try:
                        path_precio_actual_1 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--newlistPrice"][1]'
                        precio_actual_1 = driver.find_element(By.XPATH, 
                                    path_precio_actual_1).text

                        path_precio_actual_2 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--newlistPrice"][2]'
                        precio_actual_2 = driver.find_element(By.XPATH, 
                                    path_precio_actual_2).text
                    except:
                        print(f"No se pudo encontrar el precio actual {item} en el attempt {attempt}")
                        time.sleep(1)
                        continue
                    else:
                        break

                try:
                    # ================== 5. Precio anterior ================== #
                    path_precio_anterior_1 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--bestPrice"][1]'
                    precio_anterior_1 = driver.find_element(By.XPATH, 
                            path_precio_anterior_1).text

                    path_precio_anterior_2 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--bestPrice"][2]'
                    precio_anterior_2 = driver.find_element(By.XPATH, 
                            path_precio_anterior_2).text
                    # ================== 6. Descuento ================== #
                    path_descuento = f'{x_path}//p[@class="hmchile-catalog-discount-0-x-discount_grilla"]'
                    descuento = driver.find_element(By.XPATH, 
                    path_descuento).text  

                    
                except: 
                    precio_anterior_1 = 0
                    precio_anterior_2 = 0
                    descuento = 0
                

                lista_diccionarios.append({
                    'fecha': date.today(),
                    'foto': foto,
                    'item': nombre,
                    'posicion': n_item,
                    'pag': url,
                    'precio_actual': f'{precio_actual_1}{precio_actual_2}',
                    'precio_anterior': f'{precio_anterior_1}{precio_anterior_2}',
                    'genero': 'Hombre',
                    'categoria': 'Novedades',
                    'link': item,
                    'descuento': descuento
                })

            except:
                try:
                    load_more_products = './/div[@class="vtex-button__label flex items-center justify-center h-100 ph5 "]'
                    load_more_products_button = driver.find_element(By.XPATH, 
                                            load_more_products)
                    driver.execute_script("arguments[0].click();", 
                                            load_more_products_button)
                    time.sleep(5)
                except:
                    break

    return lista_diccionarios


'https://co.hm.com/1129524001/p'

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
links = [
        "https://co.hm.com/mujer/novedades",
        "https://co.hm.com/hombre/novedades", 
        ]

<ipython-input-5-e33607100bfd>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [4]:
lista_diccionarios = []
for url in links:
    n_item = 0
    driver.get(url)
    time.sleep(5)
    products_button_path = './/a[@id="menu-product"]'
    products_button = driver.find_element(By.XPATH, products_button_path)
    driver.execute_script("arguments[0].click();", products_button)
    grilla = driver.find_elements(By.XPATH, '//div[@id="gallery-layout-container"]')
    
    while True:
        try:
            # ================== Altura ================== #
            n_item += 1
            x_path = f'//div[@id="gallery-layout-container"]/div[{n_item}]'
            altura = driver.find_element("xpath", 
                        x_path).rect['y']
            driver.execute_script(f"window.scrollTo(0, {altura});")

            # ================== 1. item  ================== #
            for attempt in range(10):
                try:
                    path_item = f'{x_path}//a[@class = "vtex-product-summary-2-x-clearLink vtex-product-summary-2-x-clearLink--grilla h-100 flex flex-column"]'
                    item = driver.find_element(By.XPATH, 
                                path_item).get_attribute('href')
                except:
                    print(f"No se pudo encontrar el item {n_item} en el attempt {attempt}")
                    time.sleep(1)
                    continue
                else:
                    break
            
            # ================== 2. Foto ================== #
            for attempt in range(10):
                try:
                    path_foto = f'{x_path}//div[@class="dib relative vtex-product-summary-2-x-imageContainer vtex-product-summary-2-x-imageContainer--commonImage vtex-product-summary-2-x-imageStackContainer vtex-product-summary-2-x-hoverEffect"]/img'
                    foto = driver.find_element(By.XPATH, 
                                path_foto).get_attribute('src')
                except:
                    print(f"No se pudo encontrar la foto {item} en el attempt {attempt}")
                    time.sleep(1)
                    continue
                else:
                    break

            # ================== 3. Nombre ================== #
            for attempt in range(10):
                try:
                    path_nombre = f'{x_path}//span[@class="vtex-product-summary-2-x-productBrand vtex-product-summary-2-x-brandName t-body"]'
                    nombre = driver.find_element(By.XPATH, 
                                path_nombre).text
                except:
                    print(f"No se pudo encontrar el nombre {item} en el attempt {attempt}")
                    time.sleep(1)
                    continue
                else:
                    break

            # ================== 4. Precio actual ================== #
            for attempt in range(10):
                try:
                    path_precio_actual_1 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--newlistPrice"][1]'
                    precio_actual_1 = driver.find_element(By.XPATH, 
                                path_precio_actual_1).text

                    path_precio_actual_2 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--newlistPrice"][2]'
                    precio_actual_2 = driver.find_element(By.XPATH, 
                                path_precio_actual_2).text
                except:
                    print(f"No se pudo encontrar el precio actual {item} en el attempt {attempt}")
                    time.sleep(1)
                    continue
                else:
                    break

            try:
                # ================== 5. Precio anterior ================== #
                path_precio_anterior_1 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--bestPrice"][1]'
                precio_anterior_1 = driver.find_element(By.XPATH, 
                        path_precio_anterior_1).text

                path_precio_anterior_2 = f'{x_path}//span[@class="vtex-product-price-1-x-currencyInteger vtex-product-price-1-x-currencyInteger--bestPrice"][2]'
                precio_anterior_2 = driver.find_element(By.XPATH, 
                        path_precio_anterior_2).text
                # ================== 6. Descuento ================== #
                path_descuento = f'{x_path}//p[@class="hmchile-catalog-discount-0-x-discount_grilla"]'
                descuento = driver.find_element(By.XPATH, 
                path_descuento).text  

                
            except: 
                precio_anterior_1 = 0
                precio_anterior_2 = 0
                descuento = 0
            

            lista_diccionarios.append({
                'fecha': date.today(),
                'foto': foto,
                'item': nombre,
                'posicion': n_item,
                'pag': url,
                'precio_actual': f'{precio_actual_1}{precio_actual_2}',
                'precio_anterior': f'{precio_anterior_1}{precio_anterior_2}',
                'genero': 'Hombre',
                'categoria': 'Novedades',
                'link': item,
                'descuento': descuento
            })

        except:
            try:
                load_more_products = './/div[@class="vtex-button__label flex items-center justify-center h-100 ph5 "]'
                load_more_products_button = driver.find_element(By.XPATH, 
                                        load_more_products)
                driver.execute_script("arguments[0].click();", 
                                        load_more_products_button)
                time.sleep(5)
            except:
                break

In [6]:
lista_diccionarios = []
for url in links:
    n_item = 0
    driver.get(url)
    time.sleep(5)
    products_button_path = './/a[@id="menu-product"]'
    products_button = driver.find_element(By.XPATH, products_button_path)
    driver.execute_script("arguments[0].click();", products_button)
    grilla = driver.find_elements(By.XPATH, '//div[@id="gallery-layout-container"]')

    while True:
        try:
            # ================== Altura ================== #
            n_item += 1
            x_path = f'//div[@id="gallery-layout-container"]/div[{n_item}]'
            altura = driver.find_element("xpath", 
                        x_path).rect['y']
            driver.execute_script(f"window.scrollTo(0, {altura});")

            # ================== 1. item  ================== #
            for attempt in range(10):
                try:
                    path_item = f'{x_path}//a[@class = "vtex-product-summary-2-x-clearLink vtex-product-summary-2-x-clearLink--grilla h-100 flex flex-column"]'
                    item = driver.find_element(By.XPATH, 
                                path_item).get_attribute('href')
                except:
                    print(f"No se pudo encontrar el item {n_item} en el attempt {attempt}")
                    time.sleep(1)
                    continue
                else:
                    break

# Cambios realizados por pegar en el codigo de la pagina
        except:
            try:
                load_more_products = './/div[@class="vtex-button__label flex items-center justify-center h-100 ph5 "]'
                load_more_products_button = driver.find_element(By.XPATH, 
                                        load_more_products)
                driver.execute_script("arguments[0].click();", 
                                        load_more_products_button)
                time.sleep(5)

                for attempt in range(10):
                    try:
                        path_item = f'{x_path}//a[@class = "vtex-product-summary-2-x-clearLink vtex-product-summary-2-x-clearLink--grilla h-100 flex flex-column"]'
                        item = driver.find_element(By.XPATH, 
                                path_item).get_attribute('href')
                        break
                    except:
                        print(f"No se pudo encontrar el item {n_item} en el attempt {attempt}")
                        time.sleep(1)
                        continue
            except:
                break

            break

No se pudo encontrar el item 541 en el attempt 0
No se pudo encontrar el item 577 en el attempt 0
No se pudo encontrar el item 577 en el attempt 1
No se pudo encontrar el item 613 en el attempt 0
No se pudo encontrar el item 649 en el attempt 0
No se pudo encontrar el item 685 en el attempt 0
No se pudo encontrar el item 721 en el attempt 0
No se pudo encontrar el item 757 en el attempt 0
No se pudo encontrar el item 793 en el attempt 0
No se pudo encontrar el item 829 en el attempt 0
No se pudo encontrar el item 829 en el attempt 1
No se pudo encontrar el item 865 en el attempt 0
No se pudo encontrar el item 865 en el attempt 1
No se pudo encontrar el item 901 en el attempt 0
No se pudo encontrar el item 901 en el attempt 1
